# Exploratory notebook 2 After preliminary _qc.

Here we make a subject wise df and start cleaning using the process developed in the preliminary qc notebook. We want to start looking at RT and thinking about what kind of things will warrant subject exclusion is what we look go get out of this. 


First is loading:

In [3]:
import pandas as pd
# update to the file name to be read
# the header is the row that has the collumn names (zero indexed, and verify which is the header row before running)
# the sep is the seperator of the data
df = pd.read_csv("anagram_rating_pilot_filtered_20240808_1713.csv", header=0, sep=',')
# reading in the df from the prelim qc file
statsdf = pd.read_csv("anagram_statistics20240819_1557.csv", header=0, sep=',')

## Filter out the good trials, each trial each subject is it valid and then group by 

    1. dataframe of only anagram trials (delete the rows that aren't trials)
    2. new collumn, for each response is it valid row by row

Should be in a structure now that can be grouped by subject or by anagram

In [4]:
# similar to the qc but we will change direction 
long_df = df.filter(['workerid', 'id', 'anagram', 'response', 'rt', 'setRun']).dropna()
long_df = long_df[~((long_df['id'] == 'practice') | (long_df['id']== 'end_confirm_subjid'))]
long_df['id'] = long_df['id'].apply(lambda x: int(x))

# use the stat_df to get the correct answers using the id to match with the correct anagram, this will mean doing a lamda such that for each id in long df we get the correct
# anagram from the statsdf
long_df['correct'] = long_df.apply(lambda x: statsdf[statsdf['id'] == x['id']]['correct'].values[0], axis=1)

# Use the qc process for checking validity 
We're making a collumn that will have a boolean to check that the answer is /isn't valid. 

In [5]:
valid_responses = pd.read_csv("qc_responses.csv", header=0, sep=',') 
# function to get unique strings from each row
def get_unique_strings(row):
    # Filter out NaN values
    filtered = [item for item in row if pd.notna(item)]
    # Return the unique values in the list
    return list(set(filtered))
answers = valid_responses.apply(get_unique_strings, axis=1)
all_unique_answers = set(item for sublist in answers for item in sublist)
# make sure the whole response column is in lower case
long_df['response'] = long_df['response'].str.lower()
# make a column in the long_df to check from inside the answers if the response is valid
long_df['valid'] = long_df['response'].isin(all_unique_answers)

## Groups 
Now that there's a validity col in the long _df we can make groups to look at the data 

In [9]:
# group by the worker id and get the see the number of valid responses
worker_group = long_df.groupby('workerid')['valid'].sum()
# make the workergroup a df
worker_group = worker_group.reset_index()
# make worker_group index 
worker_index = worker_group.columns.get_loc('workerid')
# new collumn that's the rt for the valid responses for the worker_group
worker_group['rt_mean'] = long_df[long_df['valid']].groupby('workerid')['rt'].mean()
# insert the new collumn into the worker_group
# cols = worker_group.columns.tolist()
# cols.insert(worker_index +1, cols.pop(cols.index('rt_mean'))) 
# worker_group = worker_group[cols]

Always recomment the save file before a git push.

In [7]:
# from datetime import datetime
# now = datetime.now()
# dt_string = now.strftime("%Y%m%d_%H%M")
# timestamped = 'subject_stats' + dt_string + '.csv'
# df.to_csv(timestamped, index=False)
# subject_stats.drop(['response'], axis=1).to_csv(timestamped, index=False)